In [1]:
import os
import argparse
import numpy as np
import json
import csv

In [2]:
#write all tile names to a list
import os
path_albatross = 'mosaics/labeled_tiles'
image_list_albatross = [f for f in os.listdir(path_albatross) if f.endswith('.png')]
image_list_albatross

['SteepleJason_Blob_Nov2019_transparent_mosaic_group1---181.png',
 'GrandJason_SEMid1_Nov2019_transparent_mosaic_group1---35.png',
 'SteepleJason_SouthoftheNeck_Nov2019_transparent_mosaic_group1_500x500---6395.png',
 'SteepleJason_Blob_Nov2019_transparent_mosaic_group1---74.png',
 'GrandJason_SWRightThird_Nov2019_transparent_mosaic_group1_500x500---79.png',
 'SteepleJason_Bubble_Nov2019_transparent_mosaic_group1---272.png',
 'SteepleJason_Blob_Nov2019_transparent_mosaic_group1---31.png',
 'SteepleJason_Hump_Nov2019_transparent_mosaic_group1---256.png',
 'SteepleJason_Bubble_Nov2019_transparent_mosaic_group1---124.png',
 'SteepleJason_SouthoftheNeck_Nov2019_transparent_mosaic_group1_500x500---6349.png',
 'SteepleJason_SouthoftheNeck_Nov2019_transparent_mosaic_group1_500x500---6344.png',
 'SteepleJason_Bubble_Nov2019_transparent_mosaic_group1---201.png',
 'SteepleJason_Bubble_Nov2019_transparent_mosaic_group1---118.png',
 'SteepleJason_Bubble_Nov2019_transparent_mosaic_group1---234.png',

In [2]:
import os 
path_penguin = 'mosaics/labeled_tiles_penguin'
image_list_penguin = [f for f in os.listdir(path_penguin) if f.endswith('.png')]
image_list_penguin

['SteepleJason_NTiptoGully_transparent_mosaic_group1_250x250_overlap10---5937.png',
 'SteepleJason_Hump_Nov2019_transparent_mosaic_group1_500x500_overlap10---814.png',
 'SteepleJason_Hump_Nov2019_transparent_mosaic_group1_500x500_overlap10---1909.png',
 'GrandJason_SWMiddleThird_Nov2019_transparent_mosaic_group1_500x500---1623.png',
 'SteepleJason_Bubble_Nov2019_transparent_mosaic_group1_500x500_overlap20---467.png',
 'SteepleJason_Hump_Nov2019_transparent_mosaic_group1_500x500_overlap10---1115.png',
 'SteepleJason_Hump_Nov2019_transparent_mosaic_group1_500x500_overlap10---1770.png',
 'SteepleJason_Bubble_Nov2019_transparent_mosaic_group1_500x500---609.png',
 'SteepleJason_Blob_Nov2019_transparent_mosaic_group1_500x500_overlap20---132.png',
 'SteepleJason_Bubble_Nov2019_transparent_mosaic_group1_500x500---915.png',
 'GrandJason_SEColonyBlob_Nov2019_transparent_mosaic_group1_500x500---800.png',
 'GrandJason_SWMiddleThird_Nov2019_transparent_mosaic_group1_500x500---138.png',
 'SteepleJas

In [3]:
print(len(image_list_albatross))

1501


In [3]:
print(len(image_list_penguin))

1372


In [7]:
#shuffle image list and split into training, testing, validation
import random

random.shuffle(image_list_albatross)
total_count_albatross = len(image_list_albatross)

print(total_count_albatross)

test_index_albatross = int(total_count_albatross*.1)
val_index_albatross = int(total_count_albatross*.9*.1) + test_index_albatross

print(test_index_albatross, val_index_albatross)

test_data_albatross = image_list_albatross[:test_index_albatross]
val_data_albatross = image_list_albatross[test_index_albatross:val_index_albatross]
train_data_albatross = image_list_albatross[val_index_albatross:]

print(len(test_data_albatross), len(val_data_albatross), len(train_data_albatross))

1501
150 285
150 135 1216


In [4]:
#shuffle image list and split into training, testing, validation
import random

random.shuffle(image_list_penguin)
total_count_penguin = len(image_list_penguin)

print(total_count_penguin)

test_index_penguin = int(total_count_penguin*.1)
val_index_penguin = int(total_count_penguin*.9*.1) + test_index_penguin

print(test_index_penguin, val_index_penguin)

test_data_penguin = image_list_penguin[:test_index_penguin]
val_data_penguin = image_list_penguin[test_index_penguin:val_index_penguin]
train_data_penguin = image_list_penguin[val_index_penguin:]

print(len(test_data_penguin), len(val_data_penguin), len(train_data_penguin))

1372
137 260
137 123 1112


In [8]:
import csv

via_path_albatross = 'mosaics/via_labels/alltiles_latest.csv'

image_annotations_train_albatross = []
image_annotations_test_albatross = []
image_annotations_val_albatross = []

with open(via_path_albatross, "r") as f:
    reader = csv.reader(f, delimiter=",")
    for line in reader: 
        # output we want
        # path/to/image.jpg,x1,y1,x2,y2,class_name
        # /data/imgs/img_001.jpg,837,346,981,456,cow
        if '#' in line[0][0]:
            # bypassing comments in csv
            continue
        filename = line[1][2:-2]
        # strip brackets, split and get only the values we care about, then convert all the string to int 
        top_left_x, top_left_y, width, height = list(map(int,list(map(float, line[4].strip('][').split(',')[1:]))))

        if width == 0 or height == 0:
            continue
        
        # move from top left and width/height to x and y values
        if top_left_x < 0:
            top_left_x = 1
        if top_left_y < 0:
            top_left_y = 1
        x1 = top_left_x
        x2 = top_left_x + width
        y1 = top_left_y
        y2 = top_left_y + height 

        # TODO didn't add names this time since it is all one class
        name = "albatross"

        # create the csv row
        new_row = []
        if filename in train_data_albatross:
            new_row.append(filename)
            new_row.append(x1)
            new_row.append(y1)
            new_row.append(x2)
            new_row.append(y2)
            new_row.append(name)

            image_annotations_train_albatross.append(new_row)
        elif filename in test_data_albatross:
            new_row.append(filename)
            new_row.append(x1)
            new_row.append(y1)
            new_row.append(x2)
            new_row.append(y2)
            new_row.append(name)

            image_annotations_test_albatross.append(new_row)
        else:
            new_row.append(filename)
            new_row.append(x1)
            new_row.append(y1)
            new_row.append(x2)
            new_row.append(y2)
            new_row.append(name)

            image_annotations_val_albatross.append(new_row)

In [5]:
import csv

via_path_penguin = 'mosaics/via_labels/penguin_alltiles_500x500.csv'

image_annotations_train_penguin = []
image_annotations_test_penguin = []
image_annotations_val_penguin = []

with open(via_path_penguin, "r") as f:
    reader = csv.reader(f, delimiter=",")
    for line in reader: 
        # output we want
        # path/to/image.jpg,x1,y1,x2,y2,class_name
        # /data/imgs/img_001.jpg,837,346,981,456,cow
        if '#' in line[0][0]:
            # bypassing comments in csv
            continue
        filename = line[1][2:-2]
        # strip brackets, split and get only the values we care about, then convert all the string to int 
        top_left_x, top_left_y, width, height = list(map(int,list(map(float, line[4].strip('][').split(',')[1:]))))

        if width == 0 or height == 0:
            continue
        
        # move from top left and width/height to x and y values
        if top_left_x < 0:
            top_left_x = 1
        if top_left_y < 0:
            top_left_y = 1
        x1 = top_left_x
        x2 = top_left_x + width
        y1 = top_left_y
        y2 = top_left_y + height 

        # TODO didn't add names this time since it is all one class
        name = "penguin"

        # create the csv row
        new_row = []
        if filename in train_data_penguin:
            new_row.append(filename)
            new_row.append(x1)
            new_row.append(y1)
            new_row.append(x2)
            new_row.append(y2)
            new_row.append(name)

            image_annotations_train_penguin.append(new_row)
        elif filename in test_data_penguin:
            new_row.append(filename)
            new_row.append(x1)
            new_row.append(y1)
            new_row.append(x2)
            new_row.append(y2)
            new_row.append(name)

            image_annotations_test_penguin.append(new_row)
        else:
            new_row.append(filename)
            new_row.append(x1)
            new_row.append(y1)
            new_row.append(x2)
            new_row.append(y2)
            new_row.append(name)

            image_annotations_val_penguin.append(new_row)

In [9]:
print(len(image_annotations_train_albatross))
print(len(image_annotations_test_albatross))
print(len(image_annotations_val_albatross))

12431
1357
1182


In [6]:
print(len(image_annotations_train_penguin))
print(len(image_annotations_test_penguin))
print(len(image_annotations_val_penguin))

10663
1243
1285


In [37]:
image_annotations_train_total = image_annotations_train_albatross + image_annotations_train_penguin
image_annotations_test_total = image_annotations_test_albatross + image_annotations_test_penguin
image_annotations_val_total = image_annotations_val_albatross + image_annotations_val_penguin

In [38]:
print(len(image_annotations_train_total))
print(len(image_annotations_test_total))
print(len(image_annotations_val_total))

16443
2060
1461


In [7]:
with open('mosaics/labeled_tiles_penguin/test_annotations_final.csv', 'w') as fp:
    writer = csv.writer(fp)
    writer.writerows(image_annotations_test_penguin)
    
with open('mosaics/labeled_tiles_penguin/val_annotations_final.csv', 'w') as fp:
    writer = csv.writer(fp)
    writer.writerows(image_annotations_val_penguin)
    
with open('mosaics/labeled_tiles_penguin/train_annotations_final.csv', 'w') as fp:
    writer = csv.writer(fp)
    writer.writerows(image_annotations_train_penguin)

In [8]:
detection_classes = [["penguin", 0]]
with open('mosaics/labeled_tiles_penguin/classes.csv', 'w') as fp:
    writer = csv.writer(fp)
    writer.writerows(detection_classes)

In [39]:
with open('mosaics/labeled_tiles_total/test_annotations_total.csv', 'w') as fp:
    writer = csv.writer(fp)
    writer.writerows(image_annotations_test_total)
    
with open('mosaics/labeled_tiles_total/val_annotations_total.csv', 'w') as fp:
    writer = csv.writer(fp)
    writer.writerows(image_annotations_val_total)
    
with open('mosaics/labeled_tiles_total/train_annotations_total.csv', 'w') as fp:
    writer = csv.writer(fp)
    writer.writerows(image_annotations_train_total)

In [40]:
detection_classes = [["albatross", 0], ["penguin", 1]]
with open('mosaics/labeled_tiles_total/classes_total.csv', 'w') as fp:
    writer = csv.writer(fp)
    writer.writerows(detection_classes)

In [7]:
with open('mosaics/labeled_tiles/albatross_test_annotations_final.csv', 'w') as fp:
    writer = csv.writer(fp)
    writer.writerows(image_annotations_test_albatross)
    
with open('mosaics/labeled_tiles/albatross_val_annotations_final.csv', 'w') as fp:
    writer = csv.writer(fp)
    writer.writerows(image_annotations_val_albatross)
    
with open('mosaics/labeled_tiles/albatross_train_annotations_final.csv', 'w') as fp:
    writer = csv.writer(fp)
    writer.writerows(image_annotations_train_albatross)

In [21]:
detection_classes = [["albatross", 0]]
with open('mosaics/tiles/classes.csv', 'w') as fp:
    writer = csv.writer(fp)
    writer.writerows(detection_classes)